In [31]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import math
import pandas as pd
from datetime import datetime
import seaborn as sns
from sklearn.metrics import confusion_matrix
import csv

In [32]:
Error_lot_list = pd.read_csv(
    "D:\\Dataset\\#1_Dataset_장비이상 조기탐지 AI 데이터셋#1\\data\\5공정_180sec\\Error Lot list.csv",header=None,
    names=["0","1","2","3","4","5","6","7","8","9","10","11"]
)

date_load = Error_lot_list["0"][1:]
date_list = date_load.values

formatted_date_list = []

for date_str in date_list:
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    formatted_date = date_obj.strftime('%Y.%m.%d')
    formatted_date_list.append(formatted_date)

print(formatted_date_list)


['2021.09.06', '2021.09.07', '2021.09.08', '2021.09.09', '2021.09.10', '2021.09.13', '2021.09.14', '2021.09.15', '2021.09.16', '2021.09.17', '2021.09.23', '2021.09.24', '2021.09.27', '2021.09.28', '2021.09.29', '2021.09.30', '2021.10.01', '2021.10.05', '2021.10.06', '2021.10.07', '2021.10.08', '2021.10.12', '2021.10.13', '2021.10.14', '2021.10.15', '2021.10.18', '2021.10.19', '2021.10.20', '2021.10.21', '2021.10.22', '2021.10.25', '2021.10.26', '2021.10.27']


In [33]:
data = []

for date in formatted_date_list:
    csv_path = f"D:\\Dataset\\#1_Dataset_장비이상 조기탐지 AI 데이터셋#1\\data\\5공정_180sec\\kemp-abh-sensor-{date}.csv"
    df = pd.read_csv(csv_path)
    sc = df[['Temp','Current']].astype(float)
    data.append(sc)
x_label_df = pd.concat(data, axis=0,ignore_index=True)
x_label_df['Temp/Current'] = x_label_df['Temp'] / x_label_df['Current']
x_label_df = x_label_df.drop(columns=['Temp','Current'])

# 33(날짜)X43(Process)X36(Index) => 51084
print(x_label_df)


       Temp/Current
0         46.670275
1         50.104851
2         45.371934
3         44.116610
4         44.552119
...             ...
51079     43.149022
51080     44.520152
51081     43.821683
51082     45.883469
51083     45.316010

[51084 rows x 1 columns]


In [34]:
y_label=[]

for index, row in Error_lot_list.iloc[1:].iterrows():
    df = row.to_list()
    df_value = df[1:]
    for i in range(1,44):
        if i in df_value: 
            for j in range(1,37):
                y_label.append(1)
        else:
            for j in range(1,37): 
                y_label.append(0)
                
y_label_array = np.array(y_label).reshape(-1, 1)
y_label_df = pd.DataFrame(y_label_array, columns=['Y'])
print("y_label_df")
print(y_label_df)

y_label_df
       Y
0      0
1      0
2      0
3      0
4      0
...   ..
51079  0
51080  0
51081  0
51082  0
51083  0

[51084 rows x 1 columns]


In [35]:
sample_count = 3960

dataset = tf.data.Dataset.from_tensor_slices((x_label_df, y_label_df))

# y_label_df가 1인 샘플 추출
positive_samples = dataset.filter(lambda x, y: tf.math.equal(y[0], 0)).take(sample_count)

# y_label_df가 0인 샘플 추출
negative_samples = dataset.filter(lambda x, y: tf.math.equal(y[0], 1)).take(sample_count)

positive_samples = positive_samples.shuffle(buffer_size=sample_count, seed=42)
negative_samples = negative_samples.shuffle(buffer_size=sample_count, seed=42)

train_ratio = 0.6  
valid_ratio = 0.2
test_ratio = 0.2  

# 데이터셋을 분할
num_samples = 2*sample_count
train_size = int(train_ratio * num_samples)
valid_size = int(valid_ratio * num_samples)
test_size = int(test_ratio*num_samples)


positive_train_samples = positive_samples.take(train_size // 2)
negative_train_samples = negative_samples.take(train_size // 2)
train_dataset = positive_train_samples.concatenate(negative_train_samples)

positive_temp_samples = positive_samples.skip(train_size // 2)
negative_temp_samples = negative_samples.skip(train_size // 2)

positive_valid_samples = positive_temp_samples.take(valid_size // 2)
negative_valid_samples = negative_temp_samples.take(valid_size // 2)
valid_dataset = positive_valid_samples.concatenate(negative_valid_samples)

positive_test_samples = positive_temp_samples.skip(test_size // 2)
negative_test_samples = negative_temp_samples.skip(test_size // 2)
test_dataset = positive_test_samples.concatenate(negative_test_samples)



In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(x_label_df.shape[1],))
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 데이터셋 배치 설정
train_dataset = train_dataset.batch(16)
valid_dataset = valid_dataset.batch(16)

history = model.fit(train_dataset, epochs=150, validation_data=valid_dataset)
model.save("model.h5")

Epoch 1/150


297/297 [==============================] - 3s 7ms/step - loss: 1.6239 - accuracy: 0.7761 - val_loss: 0.8316 - val_accuracy: 0.5000
Epoch 2/150
297/297 [==============================] - 2s 6ms/step - loss: 0.5650 - accuracy: 0.7256 - val_loss: 0.9055 - val_accuracy: 0.5000
Epoch 3/150
297/297 [==============================] - 2s 6ms/step - loss: 0.5589 - accuracy: 0.7424 - val_loss: 0.9176 - val_accuracy: 0.5000
Epoch 4/150
297/297 [==============================] - 2s 6ms/step - loss: 0.5443 - accuracy: 0.7548 - val_loss: 0.9149 - val_accuracy: 0.5000
Epoch 5/150
297/297 [==============================] - 2s 7ms/step - loss: 0.5300 - accuracy: 0.7654 - val_loss: 0.9099 - val_accuracy: 0.5000
Epoch 6/150
225/297 [=====================>........] - ETA: 0s - loss: 0.6754 - accuracy: 0.6992 

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model

# 모델 로드
loaded_model = load_model('C:\\Users\\user\\Desktop\\test\\Define\\Define_add\\lr_model.h5')  # 모델 파일 경로로 변경

# 테스트 데이터에 대한 예측 수행
y_pred = loaded_model.predict(test_dataset)
y_pred = (y_pred > 0.5)  

# 예측 결과 출력
print("Predictions:")
print(y_pred)


1584/1584 [==============================] - 2s 1ms/step
Predictions:
[[False]
 [False]
 [False]
 ...
 [ True]
 [ True]
 [ True]]
